In [3]:
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
import random
import numpy as np
import pandas as pd
from operator import add


class DQNAgent(object):

    def __init__(self):
        self.reward = 0
        self.gamma = 0.9
        self.dataframe = pd.DataFrame()
        self.agent_target = 1
        self.agent_predict = 0
        self.learning_rate = 0.0005
        self.model = self.network()
        #self.model = self.network("weights.hdf5")
        self.epsilon = 0
        self.actual = []
        self.memory = []
        self.short_memory = []

    def get_state(self, game, player, food):

        state = [
            (player.x_change == 20 and player.y_change == 0 and ((list(map(add, player.position[-1], [20, 0])) in player.position) or
            player.position[-1][0] + 20 >= (game.game_width - 20))) or (player.x_change == -20 and player.y_change == 0 and ((list(map(add, player.position[-1], [-20, 0])) in player.position) or
            player.position[-1][0] - 20 < 20)) or (player.x_change == 0 and player.y_change == -20 and ((list(map(add, player.position[-1], [0, -20])) in player.position) or
            player.position[-1][-1] - 20 < 20)) or (player.x_change == 0 and player.y_change == 20 and ((list(map(add, player.position[-1], [0, 20])) in player.position) or
            player.position[-1][-1] + 20 >= (game.game_height-20))),  # danger straight

            (player.x_change == 0 and player.y_change == -20 and ((list(map(add,player.position[-1],[20, 0])) in player.position) or
            player.position[ -1][0] + 20 > (game.game_width-20))) or (player.x_change == 0 and player.y_change == 20 and ((list(map(add,player.position[-1],
            [-20,0])) in player.position) or player.position[-1][0] - 20 < 20)) or (player.x_change == -20 and player.y_change == 0 and ((list(map(
            add,player.position[-1],[0,-20])) in player.position) or player.position[-1][-1] - 20 < 20)) or (player.x_change == 20 and player.y_change == 0 and (
            (list(map(add,player.position[-1],[0,20])) in player.position) or player.position[-1][
             -1] + 20 >= (game.game_height-20))),  # danger right

             (player.x_change == 0 and player.y_change == 20 and ((list(map(add,player.position[-1],[20,0])) in player.position) or
             player.position[-1][0] + 20 > (game.game_width-20))) or (player.x_change == 0 and player.y_change == -20 and ((list(map(
             add, player.position[-1],[-20,0])) in player.position) or player.position[-1][0] - 20 < 20)) or (player.x_change == 20 and player.y_change == 0 and (
            (list(map(add,player.position[-1],[0,-20])) in player.position) or player.position[-1][-1] - 20 < 20)) or (
            player.x_change == -20 and player.y_change == 0 and ((list(map(add,player.position[-1],[0,20])) in player.position) or
            player.position[-1][-1] + 20 >= (game.game_height-20))), #danger left


            player.x_change == -20,  # move left
            player.x_change == 20,  # move right
            player.y_change == -20,  # move up
            player.y_change == 20,  # move down
            food.x_food < player.x,  # food left
            food.x_food > player.x,  # food right
            food.y_food < player.y,  # food up
            food.y_food > player.y  # food down
            ]

        for i in range(len(state)):
            if state[i]:
                state[i]=1
            else:
                state[i]=0

        return np.asarray(state)

    def set_reward(self, player, crash):
        self.reward = 0
        if crash:
            self.reward = -10
            return self.reward
        if player.eaten:
            self.reward = 10
        return self.reward

    def network(self, weights=None):
        model = Sequential()
        model.add(Dense(output_dim=120, activation='relu', input_dim=11))
        model.add(Dropout(0.15))
        model.add(Dense(output_dim=120, activation='relu'))
        model.add(Dropout(0.15))
        model.add(Dense(output_dim=120, activation='relu'))
        model.add(Dropout(0.15))
        model.add(Dense(output_dim=3, activation='softmax'))
        opt = Adam(self.learning_rate)
        model.compile(loss='mse', optimizer=opt)

        if weights:
            model.load_weights(weights)
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        self.short_memory.append((state, action, reward, next_state, done))

#     def replay_new(self, memory):
#         if len(memory) > 1000:
#             minibatch = random.sample(memory, 1000)
#         else:
#             minibatch = memory
#         for state, action, reward, next_state, done in minibatch:
#             target = reward
#             if not done:
#                 target = reward + self.gamma * np.amax(self.model.predict(np.array([next_state]))[0])
#             target_f = self.model.predict(np.array([state]))
#             target_f[0][np.argmax(action)] = target
#             self.model.fit(np.array([state]), target_f, epochs=1, verbose=0)
            
    def replay_new(self, memory):
        if len(self.short_memory) > 1000:
            minibatch_short = random.sample(self.short_memory, 1000)
        else:
            minibatch_short = self.short_memory
        if len(memory) > 1000:
            minibatch_long = random.sample(memory, 1000)
        else:
            minibatch_long = memory
            
        minibatch = minibatch_short + minibatch_long
            
        state_array = len(minibatch) * [None]
        target_f_array = len(minibatch) * [None]   
        index = 0
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(np.array([next_state]))[0])
            print("wow")
            target_f = self.model.predict(np.array([state]))
            print('Reward ', reward, ', Target ', target, ', Targetf ', target_f)
            target_f[0][np.argmax(action)] = target
            print('Reward ', reward, ', Target ', target, ', Targetf ', target_f)
            
            state_array[index] = np.asarray(state)
            target_f_array[index] = target_f[0]
            
            index += 1
        state_array = np.asarray(state_array)
        target_f_array = np.asarray(target_f_array)
        self.model.fit(state_array, target_f_array, epochs=1, verbose=0, batch_size = 10)
        self.short_memory = []

    def train_short_memory(self, state, action, reward, next_state, done):
        target = reward
        if not done:
            target = reward + self.gamma * np.amax(self.model.predict(next_state.reshape((1, 11)))[0])
        target_f = self.model.predict(state.reshape((1, 11)))
        target_f[0][np.argmax(action)] = target
        self.model.fit(state.reshape((1, 11)), target_f, epochs=1, verbose=0)

In [ ]:
import pygame
from random import randint
import numpy as np
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set options to activate or deactivate the game view, and its speed
display_option = False
speed = 0
pygame.font.init()


class Game:

    def __init__(self, game_width, game_height):
        pygame.display.set_caption('SnakeGen')
        self.game_width = game_width
        self.game_height = game_height
        self.gameDisplay = pygame.display.set_mode((game_width, game_height+60))
        self.bg = pygame.image.load("img/background.png")
        self.crash = False
        self.player = Player(self)
        self.food = Food()
        self.score = 0


class Player(object):

    def __init__(self, game):
        x = 0.45 * game.game_width
        y = 0.5 * game.game_height
        self.x = x - x % 20
        self.y = y - y % 20
        self.position = []
        self.position.append([self.x, self.y])
        self.food = 1
        self.eaten = False
        self.image = pygame.image.load('img/snakeBody.png')
        self.x_change = 20
        self.y_change = 0

    def update_position(self, x, y):
        if self.position[-1][0] != x or self.position[-1][1] != y:
            if self.food > 1:
                for i in range(0, self.food - 1):
                    self.position[i][0], self.position[i][1] = self.position[i + 1]
            self.position[-1][0] = x
            self.position[-1][1] = y

    def do_move(self, move, x, y, game, food,agent):
        move_array = [self.x_change, self.y_change]

        if self.eaten:

            self.position.append([self.x, self.y])
            self.eaten = False
            self.food = self.food + 1
        if np.array_equal(move ,[1, 0, 0]):
            move_array = self.x_change, self.y_change
        elif np.array_equal(move,[0, 1, 0]) and self.y_change == 0:  # right - going horizontal
            move_array = [0, self.x_change]
        elif np.array_equal(move,[0, 1, 0]) and self.x_change == 0:  # right - going vertical
            move_array = [-self.y_change, 0]
        elif np.array_equal(move, [0, 0, 1]) and self.y_change == 0:  # left - going horizontal
            move_array = [0, -self.x_change]
        elif np.array_equal(move,[0, 0, 1]) and self.x_change == 0:  # left - going vertical
            move_array = [self.y_change, 0]
        self.x_change, self.y_change = move_array
        self.x = x + self.x_change
        self.y = y + self.y_change

        if self.x < 20 or self.x > game.game_width-40 or self.y < 20 or self.y > game.game_height-40 or [self.x, self.y] in self.position:
            game.crash = True
        eat(self, food, game)

        self.update_position(self.x, self.y)

    def display_player(self, x, y, food, game):
        self.position[-1][0] = x
        self.position[-1][1] = y

        if game.crash == False:
            for i in range(food):
                x_temp, y_temp = self.position[len(self.position) - 1 - i]
                game.gameDisplay.blit(self.image, (x_temp, y_temp))

            update_screen()
        else:
            pygame.time.wait(300)


class Food(object):

    def __init__(self):
        self.x_food = 240
        self.y_food = 200
        self.image = pygame.image.load('img/food2.png')

    def food_coord(self, game, player):
        x_rand = randint(20, game.game_width - 40)
        self.x_food = x_rand - x_rand % 20
        y_rand = randint(20, game.game_height - 40)
        self.y_food = y_rand - y_rand % 20
        if [self.x_food, self.y_food] not in player.position:
            return self.x_food, self.y_food
        else:
            self.food_coord(game,player)

    def display_food(self, x, y, game):
        game.gameDisplay.blit(self.image, (x, y))
        update_screen()


def eat(player, food, game):
    if player.x == food.x_food and player.y == food.y_food:
        food.food_coord(game, player)
        player.eaten = True
        game.score = game.score + 1


def get_record(score, record):
        if score >= record:
            return score
        else:
            return record


def display_ui(game, score, record):
    myfont = pygame.font.SysFont('Segoe UI', 20)
    myfont_bold = pygame.font.SysFont('Segoe UI', 20, True)
    text_score = myfont.render('SCORE: ', True, (0, 0, 0))
    text_score_number = myfont.render(str(score), True, (0, 0, 0))
    text_highest = myfont.render('HIGHEST SCORE: ', True, (0, 0, 0))
    text_highest_number = myfont_bold.render(str(record), True, (0, 0, 0))
    game.gameDisplay.blit(text_score, (45, 440))
    game.gameDisplay.blit(text_score_number, (120, 440))
    game.gameDisplay.blit(text_highest, (190, 440))
    game.gameDisplay.blit(text_highest_number, (350, 440))
    game.gameDisplay.blit(game.bg, (10, 10))


def display(player, food, game, record):
    game.gameDisplay.fill((255, 255, 255))
    display_ui(game, game.score, record)
    player.display_player(player.position[-1][0], player.position[-1][1], player.food, game)
    food.display_food(food.x_food, food.y_food, game)


def update_screen():
    pygame.display.update()


def initialize_game(player, game, food, agent):
    state_init1 = agent.get_state(game, player, food)  # [0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0]
    action = [1, 0, 0]
    player.do_move(action, player.x, player.y, game, food, agent)
    state_init2 = agent.get_state(game, player, food)
    reward1 = agent.set_reward(player, game.crash)
    agent.remember(state_init1, action, reward1, state_init2, game.crash)
    agent.replay_new(agent.memory)


def plot_seaborn(array_counter, array_score):
    sns.set(color_codes=True)
    ax = sns.regplot(np.array([array_counter])[0], np.array([array_score])[0], color="b", x_jitter=.1, line_kws={'color':'green'})
    ax.set(xlabel='games', ylabel='score')
    plt.show()

def run():
    pygame.init()
    agent = DQNAgent()
    counter_games = 0
    score_plot = []
    counter_plot =[]
    record = 0
    while counter_games < 150:
        # Initialize classes
        game = Game(440, 440)
        player1 = game.player
        food1 = game.food

        # Perform first move
        initialize_game(player1, game, food1, agent)
        if display_option:
            display(player1, food1, game, record)

        while not game.crash:
            #agent.epsilon is set to give randomness to actions
            agent.epsilon = 80 - counter_games
            
            #get old state
            state_old = agent.get_state(game, player1, food1)
            
            #perform random actions based on agent.epsilon, or choose the action
            if randint(0, 200) < agent.epsilon:
                final_move = to_categorical(randint(0, 2), num_classes=3)
            else:
                # predict action based on the old state
                prediction = agent.model.predict(state_old.reshape((1,11)))
                final_move = to_categorical(np.argmax(prediction[0]), num_classes=3)
                
            #perform new move and get new state
            player1.do_move(final_move, player1.x, player1.y, game, food1, agent)
            state_new = agent.get_state(game, player1, food1)
            
            #set treward for the new state
            reward = agent.set_reward(player1, game.crash)
            
            #train short memory base on the new action and state
            #agent.train_short_memory(state_old, final_move, reward, state_new, game.crash)
            
            # store the new data into a long term memory
            agent.remember(state_old, final_move, reward, state_new, game.crash)
            record = get_record(game.score, record)
            if display_option:
                display(player1, food1, game, record)
                pygame.time.wait(speed)
        
        agent.replay_new(agent.memory)
        counter_games += 1
        print('Game', counter_games, '      Score:', game.score)
        score_plot.append(game.score)
        counter_plot.append(counter_games)
    agent.model.save_weights('weights.hdf5')
    plot_seaborn(counter_plot, score_plot)


run()

wow
Reward  0 , Target  0.3196378290653229 , Targetf  [[0.35515314 0.30743483 0.33741206]]
Reward  0 , Target  0.3196378290653229 , Targetf  [[0.31963784 0.30743483 0.33741206]]
wow
Reward  0 , Target  0.3196378290653229 , Targetf  [[0.35515314 0.30743483 0.33741206]]
Reward  0 , Target  0.3196378290653229 , Targetf  [[0.31963784 0.30743483 0.33741206]]
Instructions for updating:
Use tf.cast instead.


C:\Users\logan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:79: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=11, units=120)`
C:\Users\logan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:81: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=120)`
C:\Users\logan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:83: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=120)`
C:\Users\logan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:85: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=3)`


wow
Reward  0 , Target  0.30987405180931094 , Targetf  [[0.3443045  0.31298468 0.34271082]]
Reward  0 , Target  0.30987405180931094 , Targetf  [[0.30987406 0.31298468 0.34271082]]
wow
Reward  0 , Target  0.30723789632320403 , Targetf  [[0.3443045  0.31298468 0.34271082]]
Reward  0 , Target  0.30723789632320403 , Targetf  [[0.3072379  0.31298468 0.34271082]]
wow
Reward  10 , Target  10.317156571149827 , Targetf  [[0.3384107  0.32021385 0.34137544]]
Reward  10 , Target  10.317156571149827 , Targetf  [[ 0.3384107   0.32021385 10.317157  ]]
wow
Reward  0 , Target  0.31715657114982604 , Targetf  [[0.3523962  0.31283838 0.33476546]]
Reward  0 , Target  0.31715657114982604 , Targetf  [[0.31715658 0.31283838 0.33476546]]
wow
Reward  0 , Target  0.33501053452491764 , Targetf  [[0.3523962  0.31283838 0.33476546]]
Reward  0 , Target  0.33501053452491764 , Targetf  [[0.3523962  0.33501053 0.33476546]]
wow
Reward  0 , Target  0.34733453392982483 , Targetf  [[0.37223393 0.30789626 0.31986982]]
Rewar

Reward  0 , Target  0.32105891704559325 , Targetf  [[0.31173876 0.3315291  0.35673213]]
Reward  0 , Target  0.32105891704559325 , Targetf  [[0.32105893 0.3315291  0.35673213]]
wow
Reward  0 , Target  0.32105891704559325 , Targetf  [[0.31173876 0.3315291  0.35673213]]
Reward  0 , Target  0.32105891704559325 , Targetf  [[0.32105893 0.3315291  0.35673213]]
wow
Reward  0 , Target  0.312523028254509 , Targetf  [[0.31173876 0.3315291  0.35673213]]
Reward  0 , Target  0.312523028254509 , Targetf  [[0.31252304 0.3315291  0.35673213]]
wow
Reward  -10 , Target  -10 , Targetf  [[0.3120369 0.3407153 0.3472478]]
Reward  -10 , Target  -10 , Targetf  [[-10.          0.3407153   0.3472478]]
wow
Reward  0 , Target  0.34404794275760653 , Targetf  [[0.30206364 0.31566092 0.3822755 ]]
Reward  0 , Target  0.34404794275760653 , Targetf  [[0.34404793 0.31566092 0.3822755 ]]
wow
Reward  0 , Target  0.3183463990688324 , Targetf  [[0.30206364 0.31566092 0.3822755 ]]
Reward  0 , Target  0.3183463990688324 , Targ

Reward  -10 , Target  -10 , Targetf  [[  0.309048     0.34214625 -10.        ]]
wow
Reward  0 , Target  0.359346753358841 , Targetf  [[0.2871824  0.3135434  0.39927417]]
Reward  0 , Target  0.359346753358841 , Targetf  [[0.35934675 0.3135434  0.39927417]]
wow
Reward  0 , Target  0.3141389787197113 , Targetf  [[0.2817928  0.31396645 0.40424082]]
Reward  0 , Target  0.3141389787197113 , Targetf  [[0.2817928  0.31396645 0.31413898]]
wow
Reward  0 , Target  0.3161122262477875 , Targetf  [[0.31894052 0.33201617 0.3490433 ]]
Reward  0 , Target  0.3161122262477875 , Targetf  [[0.31894052 0.33201617 0.31611222]]
wow
Reward  0 , Target  0.3100998133420944 , Targetf  [[0.30719638 0.3512358  0.3415678 ]]
Reward  0 , Target  0.3100998133420944 , Targetf  [[0.30719638 0.3100998  0.3415678 ]]
wow
Reward  0 , Target  0.32544065415859225 , Targetf  [[0.316883   0.33856162 0.34455535]]
Reward  0 , Target  0.32544065415859225 , Targetf  [[0.316883   0.33856162 0.32544065]]
wow
Reward  0 , Target  0.3100

...

Reward  0 , Target  0.6333723843097687 , Targetf  [[0.09514274 0.2789193  0.625938  ]]
Reward  0 , Target  0.6333723843097687 , Targetf  [[0.09514274 0.2789193  0.63337237]]
wow
Reward  0 , Target  0.5999540984630585 , Targetf  [[0.07805569 0.21819726 0.7037471 ]]
Reward  0 , Target  0.5999540984630585 , Targetf  [[0.07805569 0.21819726 0.5999541 ]]
wow
Reward  0 , Target  0.5209564447402955 , Targetf  [[0.10025068 0.23313366 0.66661566]]
Reward  0 , Target  0.5209564447402955 , Targetf  [[0.10025068 0.23313366 0.52095646]]
wow
Reward  0 , Target  0.5633441984653473 , Targetf  [[0.11173996 0.30941954 0.5788405 ]]
Reward  0 , Target  0.5633441984653473 , Targetf  [[0.11173996 0.30941954 0.5633442 ]]
wow
Reward  0 , Target  0.6333723843097687 , Targetf  [[0.09514274 0.2789193  0.625938  ]]
Reward  0 , Target  0.6333723843097687 , Targetf  [[0.09514274 0.2789193  0.63337237]]
wow
Reward  0 , Target  0.5999540984630585 , Targetf  [[0.07805569 0.21819726 0.7037471 ]]
Reward  0 , Target  0.5

wow
Reward  0 , Target  0.4180067181587219 , Targetf  [[0.19061686 0.31899923 0.4903839 ]]
Reward  0 , Target  0.4180067181587219 , Targetf  [[0.19061686 0.41800672 0.4903839 ]]
wow
Reward  0 , Target  0.4413455039262772 , Targetf  [[0.19980766 0.33574045 0.4644519 ]]
Reward  0 , Target  0.4413455039262772 , Targetf  [[0.19980766 0.33574045 0.4413455 ]]
wow
Reward  0 , Target  0.388292133808136 , Targetf  [[0.19061686 0.31899923 0.4903839 ]]
Reward  0 , Target  0.388292133808136 , Targetf  [[0.19061686 0.31899923 0.38829213]]
wow
Reward  0 , Target  0.38507015705108644 , Targetf  [[0.21736005 0.3512042  0.4314357 ]]
Reward  0 , Target  0.38507015705108644 , Targetf  [[0.21736005 0.3512042  0.38507015]]
wow
Reward  0 , Target  0.38507015705108644 , Targetf  [[0.23547348 0.3366708  0.42785573]]
Reward  0 , Target  0.38507015705108644 , Targetf  [[0.38507015 0.3366708  0.42785573]]
wow
Reward  0 , Target  0.4180067181587219 , Targetf  [[0.23547348 0.3366708  0.42785573]]
Reward  0 , Targe

Reward  0 , Target  0.4937136769294739 , Targetf  [[0.13789098 0.27020454 0.59190446]]
Reward  0 , Target  0.4937136769294739 , Targetf  [[0.13789098 0.27020454 0.49371368]]
wow
Reward  0 , Target  0.5327140152454376 , Targetf  [[0.14473926 0.30668998 0.54857075]]
Reward  0 , Target  0.5327140152454376 , Targetf  [[0.14473926 0.532714   0.54857075]]
wow
Reward  0 , Target  0.6311538219451904 , Targetf  [[0.13789098 0.27020454 0.59190446]]
Reward  0 , Target  0.6311538219451904 , Targetf  [[0.13789098 0.6311538  0.59190446]]
wow
Reward  0 , Target  0.6311538219451904 , Targetf  [[0.09640029 0.20231766 0.701282  ]]
Reward  0 , Target  0.6311538219451904 , Targetf  [[0.6311538  0.20231766 0.701282  ]]
wow
Reward  0 , Target  0.5327140152454376 , Targetf  [[0.09640029 0.20231766 0.701282  ]]
Reward  0 , Target  0.5327140152454376 , Targetf  [[0.09640029 0.20231766 0.532714  ]]
wow
Reward  0 , Target  0.6311538219451904 , Targetf  [[0.13789098 0.27020454 0.59190446]]
Reward  0 , Target  0.6

wow
Reward  0 , Target  0.6311538219451904 , Targetf  [[0.13789098 0.27020454 0.59190446]]
Reward  0 , Target  0.6311538219451904 , Targetf  [[0.13789098 0.6311538  0.59190446]]
wow
Reward  0 , Target  0.5327140152454376 , Targetf  [[0.09640029 0.20231766 0.701282  ]]
Reward  0 , Target  0.5327140152454376 , Targetf  [[0.09640029 0.20231766 0.532714  ]]
wow
Reward  0 , Target  0.4937136769294739 , Targetf  [[0.13789098 0.27020454 0.59190446]]
Reward  0 , Target  0.4937136769294739 , Targetf  [[0.13789098 0.27020454 0.49371368]]
wow
Reward  0 , Target  0.5387515068054199 , Targetf  [[0.14473926 0.30668998 0.54857075]]
Reward  0 , Target  0.5387515068054199 , Targetf  [[0.14473926 0.30668998 0.5387515 ]]
wow
Reward  0 , Target  0.6311538219451904 , Targetf  [[0.12503222 0.2763549  0.5986128 ]]
Reward  0 , Target  0.6311538219451904 , Targetf  [[0.12503222 0.2763549  0.6311538 ]]
wow
Reward  0 , Target  0.5327140152454376 , Targetf  [[0.09640029 0.20231766 0.701282  ]]
Reward  0 , Target 

wow
Reward  0 , Target  0.5327140152454376 , Targetf  [[0.13789098 0.27020454 0.59190446]]
Reward  0 , Target  0.5327140152454376 , Targetf  [[0.532714   0.27020454 0.59190446]]
wow
Reward  0 , Target  0.4937136769294739 , Targetf  [[0.13789098 0.27020454 0.59190446]]
Reward  0 , Target  0.4937136769294739 , Targetf  [[0.13789098 0.27020454 0.49371368]]
wow
Reward  0 , Target  0.4937136769294739 , Targetf  [[0.14473926 0.30668998 0.54857075]]
Reward  0 , Target  0.4937136769294739 , Targetf  [[0.49371368 0.30668998 0.54857075]]
wow
Reward  0 , Target  0.5387515068054199 , Targetf  [[0.14473926 0.30668998 0.54857075]]
Reward  0 , Target  0.5387515068054199 , Targetf  [[0.14473926 0.30668998 0.5387515 ]]
wow
Reward  0 , Target  0.4937136769294739 , Targetf  [[0.12503222 0.2763549  0.5986128 ]]
Reward  0 , Target  0.4937136769294739 , Targetf  [[0.12503222 0.49371368 0.5986128 ]]
wow
Reward  0 , Target  0.5387515068054199 , Targetf  [[0.14473926 0.30668998 0.54857075]]
Reward  0 , Target 